## Exercise 1: Introduction 🤖

> Welcome to the Advanced Robot Learning and Decision Making exercises!

 This first exercise will guide you through concepts and tools that we will use throughout the course. By the end of this exercise, you will have a foundational understanding of:

1. **Vector Computations in Python (NumPy)**

   A brief introduction to NumPy for handling vector and matrix computations, which are fundamental for scientific computations.

2. **Introduction to Gymnasium environments** 

   We will explore the Gymnasium API, a standard API for control and reinforcement learning environments. Gymnasium enables us to develop controllers that can be used with a variety of simulations.

3. **Introduction to the Drone Simulation and Environment**

   We will introduce you to the Drone environment, which we will use throughout the exercise. The drone simulation already implements input constraints that you would experience on hardware.

4. **Design of a simple PD controller for the Drone**

   We will implement a simple PD controller that lets the drone hover at a target position.

5. **Testing and Submission-System**

   Finally, an example task to get familiar with the tests and submission system.

## 1. Introduction to NumPy

[NumPy](https://numpy.org) is a powerful library for numerical scientific computations in Python. It provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these data structures efficiently.

**It is important that you understand the concepts of NumPy conveyed in this exercise, as we will use NumPy heavily in the coming exercises.** Feel free to play around with the NumPy functions in this notebook. Many simulation in robotics are written in NumPy, or its brothers [PyTorch](https://pytorch.org) and [JAX](https://docs.jax.dev) - more on that later.

### 1.1 Why Use NumPy?

#### 1.1.1 Vector Operations

Using Python lists for mathematical operations requires explicit loops, which can be cumbersome and error-prone. NumPy allows for element-wise operations without the need for explicit iteration. If you want to implement a simple vector addition

$$
\mathbf{a} + \mathbf{b} =
\begin{pmatrix}
a_1 \\ a_2 \\ a_3
\end{pmatrix} +
\begin{pmatrix}
b_1 \\ b_2 \\ b_3
\end{pmatrix} =
\begin{pmatrix}
a_1 + b_1 \\ a_2 + b_2 \\ a_3 + b_3
\end{pmatrix}
$$

with Python lists, you would need to loop over each element. NumPy on the other hand allows for element-wise operations without the need for explicit iteration.

The benefits of using numpy's vectorized computations compared to standard python operations is enormous!


In [ ]:
# magic-commands to enable auto-reloading of external modules
%load_ext autoreload
%autoreload 2

# imports
import numpy as np

In [ ]:
# Using Python lists
a = [1, 2, 3]
b = [4, 5, 6]
c_list = [a[i] + b[i] for i in range(len(a))]  # Explicit loop required

# Using NumPy
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c_numpy = a + b  # No loop, but vectorized computation. Faster for large arrays and more readable

# Both results are the same
assert np.allclose(c_numpy, c_list), "The results are not the same"


#### 1.1.2 Performance

NumPy operations are implemented in C and optimized for performance. This makes them significantly faster than equivalent operations using Python lists. Below you can find an example that produces the same results with Python and NumPy, but will run up to 50x faster in NumPy depending on your machine. Of course, this example is rather trivial. In more complex scenarios, NumPy can accelerate your code by several magnitudes.

In [ ]:
import time

a = list(range(int(1e6)))
b = list(range(int(1e6)))
start = time.perf_counter()
c = [a[i] + b[i] for i in range(len(a))]  # loop
end = time.perf_counter()
duration_list = end - start
print(f"Python list addition time: {duration_list:.2e} seconds")

# NumPy equivalent
an = np.array(a)
bn = np.array(b)
start = time.perf_counter()
cn = an + bn  # vectorized
end = time.perf_counter()
duration_numpy = end - start
print(f"NumPy addition time: {duration_numpy:.2e} seconds")

print(f"NumPy speed up: {duration_list / duration_numpy:.2f}x")

#### 1.1.3 Basic NumPy Operations

NumPy includes many vectorized vector, matrix and other linear algebra operations out of the box. You can find them [in the numpy docs](https://numpy.org/doc/stable/reference/routines.html). Have a quick looks at the docs to get a feeling of available functions. To give you some examples:

In [ ]:
x = np.array([1, 2, 3])
y = np.array([4, 5, 6])

# Element-wise addition
z = x + y
print(f"Vector addition: {z}")

# Element-wise multiplication
z = x * y
print(f"Element-wise multiplication: {z}")

# Dot product
z = np.dot(x, y)
print(f"Dot product: {z}")

# Matrix multiplication
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
z = np.dot(A, B)  # Or A @ B
print(f"Matrix multiplication: \n{z}")

# Transpose
print(f"A: \n{A}\nA.T: \n{A.T}")

### 1.2 Broadcasting in NumPy

[Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) is a powerful feature in NumPy that allows operations between arrays of different shapes by automatically expanding them to a compatible shape.

**Rules of Broadcasting**

1. If the arrays have different dimensions, the smaller array is padded with leading dimensions of size 1.
2. If the shape of the arrays does not match in any dimension, the array with size 1 in that dimension is stretched to match the other array.
3. If the dimensions are incompatible (i.e., neither is 1 and they are different), NumPy throws an error.

**Why is Broadcasting Useful?**

Broadcasting allows for efficient vectorized operations, eliminating the need for explicit loops and improving performance.

Examples:


In [ ]:
# Broadcasting across all elements
A = np.array([[1, 2], [3, 4]])
b = 5
print(f"A + b: \n{A + b}")  # Each element of A is incremented by 5

In [ ]:
# Broadcasting across rows
A = np.array([[1, 2], [3, 4]])
b = np.array([5, 6])
print(f"A + b: \n{A + b}")  # Broadcasting across rows

In [ ]:
A = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
print(f"A + b.T: \n{A + b.T}")  # Broadcasting across columns

Of course, these examples only scratch the surface of what NumPy can do. If you want to find out more, you can look up details of specific functions and much more in their [documentation](https://numpy.org/doc/stable/index.html).

It is good to know that brother libraries such as [PyTorch](https://pytorch.org) and [JAX](https://docs.jax.dev) often have very similar syntax to NumPy!

Equipped with some basic knowledge on how to handle linear algebra operations, we can now take a look at the de-facto standard API for learning and control with Python.


## 2. The Gymnasium API

The following is quite some text, but it allows you to gain a background understanding of the framework we use.

### 2.1 Background

The core idea behind Gymnasium is to provide an [API](https://en.wikipedia.org/wiki/API) for all (single agent)reinforcement learning environments. It also provides a set of (rather simple) environments out of the box that are typically used for benchmarking new algorithms: cartpole, pendulum, mujoco (walker, humanoid, ...), atari, and more. The Gymnasium API is de facto becomgin the standard for projects that are used with control algorithms. Often, Gymnasium environments function as an interface (API) between a lower-level simulation and the user-implemented controller. This is, for instance, the case for MuJoCo environments, such as [Gymnasium Robotics](https://robotics.farama.org/index.html) where the simulation is done using [MuJoCo](https://mujoco.org/), or custom environments, such as our [Crazyflow environment](https://github.com/utiasDSL/crazyflow) where simulation is done using [JAX](https://docs.jax.dev/en/latest/quickstart.html) and [MJX](https://mujoco.readthedocs.io/en/stable/mjx.html), and many more. The Gymnasium Environments implement task-specific details, such as reward functions and termination conditions.

At the core of Gymnasium is [`Env`](https://gymnasium.farama.org/api/env/#gymnasium.Env), a high-level python class representing a [markov decision process (MDP)](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#optional-formalism) from reinforcement learning theory. The class provides users the ability to generate an initial state, transition to new states given an action, and visualize the environment. Alongside Env, [Wrappers](https://gymnasium.farama.org/api/wrappers/#gymnasium.Wrapper) are provided to help augment and modify the environment for particular tasks, for instance the agent observations, rewards and actions taken.

>**Note** While Gymnasium was originally developed for reinforcement learning, we will use the same interface for other control implementations.

### [2.2 Basic Usage](https://gymnasium.farama.org/introduction/basic_usage/#basic-usage)

Initializing environments is very easy in Gymnasium and can be done via the [`make()`](https://gymnasium.farama.org/api/registry/#gymnasium.make) function:

```python
import gymnasium as gym
env = gym.make('CartPole-v1')
```
This function will return an Env for users to interact with (in that case the env with the name `CartPole-v1`). Furthermore, `make()` can provid a number of additional arguments for specifying keywords to the environment, adding more or less wrappers, etc. You will see examples of that later.


The classic “agent-environment loop” pictured below is a simplified representation of how an agent and environment interact with each other. The agent receives an observation about the environment, the agent then selects an action (usually using a controller), which the environment uses to determine the reward and the next observation. The cycle then repeats itself until the environment ends.

# ![Agent-Environment Loop](./img/AE_loop_dark.png)


For Gymnasium, the “agent-environment-loop” is implemented below.

```python
import gymnasium as gym

env = gym.make("LunarLander-v3", render_mode="human")
observation, info = env.reset() # get initial observation

episode_over = False
while not episode_over:
    action = env.action_space.sample()  # randomly sample an action; in practise one would use a controller here
    observation, reward, terminated, truncated, info = env.step(action) # step the environment one timestep while applying the action, and return the environment information

    episode_over = terminated or truncated
    # if one wants to continue to step through the environment when episode_over == True, one must call env.reset()

env.close()
```

After initializing the environment, we **must** call [`env.reset()`](https://gymnasium.farama.org/api/env/#gymnasium.Env.reset) to get the first observation of the environment along with optional additional information. For initializing the environment with a particular random seed or options you can pass those parameters to `reset()` - please refer to the documentation.

Next, the agent performs an action in the environment, [`env.step()`](https://gymnasium.farama.org/api/env/#gymnasium.Env.step) executes the selected action (in this case random with `env.action_space.sample()`) to update the environment. This action can be imagined as moving a robot or pressing a button on a games’ controller that causes a change within the environment. As a result, the agent receives a new observation from the updated environment along with a reward for taking the action. This reward could be for instance positive for destroying an enemy or a negative reward for moving into lava. One such action-observation exchange is referred to as a timestep.

After some timesteps, the environment may end, this is called the **terminal state**. For instance, the robot may have crashed, or may have succeeded in completing a task, the environment will need to stop as the agent cannot continue. In Gymnasium, if the environment has **terminated**, this is returned by `step()` as the third variable. Similarly, we may also want the environment to end after a fixed number of timesteps, in this case, the environment issues a **truncated** signal. If either of terminated or truncated are True then we end the episode but **in most cases users might wish to restart the environment, this can be done with `env.reset()`**.

**There are some important environment design factors that we need you to understand for the following exercises**:
- [Observation space](https://gymnasium.farama.org/api/env/#gymnasium.Env.observation_space): All valid observations are contained in that space. For control applications, we most commonly use [Box](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Box).
- [Action space](https://gymnasium.farama.org/api/env/#gymnasium.Env.action_space): All valid actions must be in that space. For control applications, again [Box](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Box) is most commonly used, or [Discrete](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Discrete).
- **Truncate**: Most environments are set to have a time limit of a certain number of timesteps, which is usually implemented using the [TimeLimit wrapper](https://gymnasium.farama.org/api/wrappers/misc_wrappers/#gymnasium.wrappers.TimeLimit). The environment will truncate if the time limit is reached.
- **Terminate**: Environments terminate after reaching a termination condition, such as a drone crashing or reaching its final goal.
- If any of truncate or terminate occures, it is returned by the [step function](https://gymnasium.farama.org/api/env/#gymnasium.Env.step) as shown above. In that case you usually reset the environment.
- **Reset**: Environments are usually reset to a random state, which help to test the robustness of control algorithms. You can make the random state reproducibly by passing a [seed](https://en.wikipedia.org/wiki/Random_seed) to a [reset function](https://gymnasium.farama.org/api/env/#gymnasium.Env.reset). (Seeding is a common concepts for random number generators - it is useful to read up on this if you haven't done so.) Some environments, such as [crazyflow, also allow to set the initial state explicitely](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py#L150), which is useful during development of control algorithms.

>**Note**: if you come from a control background, you already know about the **state** of a system. A state must not be confused with **observations**. Typically, the observation is a subspace of the state that the policy receives as input to calculate the actions. An observation can also be an RGB image obtained from a camera. Its important to understand the [difference between states and observations](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#states-and-observations).

## 3 Drone simulation and environment
Throughout the semester, you will develop controllers for a simulation of the [Crazyflie 2.1](https://www.bitcraze.io/products/old-products/crazyflie-2-1/). Here, we introduce the dynamics model, the simulation, and gymnasium environment.

##### Drone dynamics model

 We use the following second-order time-continuous dynamics model of the drone. The state $x$ is the position of its center of mass $(x, y, z)$, the body orientations roll, pitch, and yaw $(\phi, \theta, \psi)$, and their respective derivatives. The control inputs $u$ are the vertical thrust $F_z$ and the desired roll pitch yaw angles $(\phi_d, \theta_d, \psi_d)$ around each axis of a coordinate frame fixed to the quadrotor body. This is a so-called [attitude control interface](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/functional-areas/sensor-to-control/controllers/). $g$ is gravity.
$$
\begin{equation} \tag{1} 
    \frac{d}{dt}\begin{bmatrix} x\\ y\\ z\\ \phi\\ \theta\\ \psi\\ \dot{x}\\ \dot{y}\\ \dot{z}\\ \dot{\phi}\\ \dot{\theta}\\ \dot{\psi} \end{bmatrix}
    = 
    \begin{bmatrix} \dot{x}\\ \dot{y}\\ \dot{z}\\ \dot{\phi}\\ \dot{\theta}\\ \dot{\psi}\\ 0\\ 0\\ -g\\ \alpha_{\phi}\phi + \gamma_{\phi}\dot{\phi}\\ \alpha_{\theta}\theta + \gamma_{\theta}\dot{\theta}\\ \alpha_{\psi}\psi + \gamma_{\psi}\dot{\psi} \end{bmatrix} 
    + 
    \begin{bmatrix} 
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0\\
    \frac{1}{m} (\cos\phi \sin\theta \cos\psi + \sin\phi \sin\psi) & 0 & 0 & 0\\ 
    \frac{1}{m} (\cos\phi \sin\theta \sin\psi - \sin\phi \cos\psi) & 0 & 0 & 0\\ 
    \frac{1}{m} \cos\phi \cos\theta & 0 & 0 & 0\\
    0 & \beta_{\phi} & 0 & 0\\
    0 & 0 & \beta_{\theta} & 0\\
    0 & 0 & 0 & \beta_{\psi}
    \end{bmatrix} u, 
\end{equation}
$$
with
$$ 
\begin{equation} \tag{2}
    u = (F_z, \phi_d, \theta_d, \psi_d).
\end{equation}
$$

The parameters $\alpha, \beta, \gamma, m$ of the model have been determined using system identification. For this, we collected data and fitted the parameters using a least-square error.

In a general form, such a system can be written as


$$
\begin{equation} \tag{3}
    \dot{x} = f(x) + g(x)\cdot u.
\end{equation}
$$



For the simulation on a computer, we require a time-discretized version of the model for each timestep $k$ that approximates the next state:
$$
\begin{align} \tag{4}
    \frac{x_{x+1}-x_k}{\delta t} &= f(x_k) + g(x_k) \cdot u_k \\
    \tag{5}
    x_{k+1} &= \delta t (f(x_k) + g(x_k) \cdot u_k) + x_k
\end{align}
$$

Equation 5) can be used to step the dynamics model in time on a computer. There are different methods to solve this integration step. More advanced methods, such as RK4, approximate the gradient during $\delta t$ more precisely.

We implement this discretized version of the model in a simulation that we call [`crazyflow`](https://github.com/utiasDSL/crazyflow). This simulation is optimized for speed and can run highly-efficient parallalized on the GPU.

Additionally, we provide the time-continuous model as a [CasADi](https://web.casadi.org/) function. CasADi is a general-purpose tool for gradient-based numerical optimization – with a strong focus on optimal control. It implement algorithmic differentiation (AD), but also tools for numerical integration. Internal CasADi also uses an integrator to discretize the model.


##### Simulation

An efficient, highly-parallizable (GPU) simulation of the discretized dynamics models is implemented in our simulator [`crazyflow`](https://github.com/utiasDSL/crazyflow).

- The simulation is implemented in [JAX](https://docs.jax.dev/en/latest/quickstart.html), which is basically numpy for the GPU, and enables us to massively parallize the simulation.
- The simulation provides physics, integrators (Euler or RK4), lower level controllers, and visualisation routines. If you are interested, you can check out the [this main function](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/sim/sim.py) of the simulation.
- The default simulation step-size is $2\text{ms}$. The simulation provides Explicit Euler or RK4 for integration. We recommend to stick with the values that have been set for this exercise.
- For visualisation, we use [MuJoCo](https://mujoco.org/), or more specific [MJX](https://mujoco.readthedocs.io/en/stable/mjx.html). While MuJoCo also comes with its own physics engine, we only use it for vizualization in crazyflow.

> **Note**: Crazyflow is of course already installed in the container. You can find the code in `/home/vscode/venv/lib/python3.11/site-packages/crazyflow`, but **do not make any changes**. For the exercises, we use [`crazyflow v0.0.2`](https://github.com/utiasDSL/crazyflow/tree/v0.0.2).



##### Gymnasium environment

The simulation is wrapped with different [Gymnasium environments](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py) that define different tasks. You will use the environments to interact with the drone simulation using your controllers, as explained in the previous section. We provide [multiple environments](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py), each for a different purpose. The environments differ in their goals (reward formulations), observation spaces, and other details. For instance, we have `CrazyflowEnvReachGoal` and `CrazyflowEnvFigureEightTrajectory`. We will use the different environments throughout the exercises.
- **Observation space**: The observation spaces differ for the different gymnasium environments. However, each observation includes at least the state of the drone, which are `position`, `orientation` (in quaternion), and its derivatives. Some environments include additional observations, e.g. the `CrazyflowEnvReachGoal` additionally includes the `difference_to_goal`.
- **Action space** (corresponds to control input $u$): We use the [Attitude controll interface](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/functional-areas/sensor-to-control/controllers/) of the drone, as specified in the dynamics model above. This interface takes **four control inputs**: `[collective thrust, roll, pitch, yaw]`. Thus, the action space is a [continuous box](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Box) represented as <b>float32</b>. The range for the thrust is <b>0.11264675 to 0.5933658</b>, while the orientations range from <b>-π to π</b>. Actions needs to be a 2D array of shape `(n_envs, n_actions)`. For all exercises except the last one, we always have `n_envs=1` and `n_actions=4`.


> **Note**: There are different ways to represent orientations in 3D space. The observations of the environment returns [quaternions](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation), while the symbolic model works with Euler Angles. You do not have to worry too much about that, as you will use [this scipy package](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) to easily convert orientation representations.

> **Note**: The actions need to be a 2D array of shape `(n_envs, n_actions)`. Except for parallel simulations on GPU for learning based methods, we set `n_envs=1`.

##### Simulation

An efficient, highly-parallizable (GPU) simulation of the discretized dynamics models is implemented in our simulator [`crazyflow`](https://github.com/utiasDSL/crazyflow).

- The simulation is implemented in [JAX](https://docs.jax.dev/en/latest/quickstart.html), which is basically numpy for the GPU, and enables us to massively parallize the simulation.
- The simulation provides physics, integrators (Euler or RK4), lower level controllers, and visualisation routines. If you are interested, you can check out the [this main function](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/sim/sim.py) of the simulation.
- The default simulation step-size is $2\text{ms}$. The simulation provides Explicit Euler or RK4 for integration. We recommend to stick with the values that have been set for this exercise.
- For visualisation, we use [MuJoCo](https://mujoco.org/), or more specific [MJX](https://mujoco.readthedocs.io/en/stable/mjx.html). While MuJoCo also comes with its own physics engine, we only use it for vizualization in crazyflow.

> **Note**: Crazyflow is of course already installed in the container. You can find the code in `/home/vscode/venv/lib/python3.11/site-packages/crazyflow`, but **do not make any changes**. For the exercises, we use [`crazyflow v0.0.2`](https://github.com/utiasDSL/crazyflow/tree/v0.0.2).



##### Gymnasium environment

The simulation is wrapped with different [Gymnasium environments](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py) that define different tasks. You will use the environments to interact with the drone simulation using your controllers, as explained in the previous section. We provide [multiple environments](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py), each for a different purpose. The environments differ in their goals (reward formulations), observation spaces, and other details. For instance, we have `CrazyflowEnvReachGoal` and `CrazyflowEnvFigureEightTrajectory`. We will use the different environments throughout the exercises.
- **Observation space**: The observation spaces differ for the different gymnasium environments. However, each observation includes at least the state of the drone, which are `position`, `orientation` (in quaternion), and its derivatives. Some environments include additional observations, e.g. the `CrazyflowEnvReachGoal` additionally includes the `difference_to_goal`.
- **Action space** (corresponds to control input $u$): We use the [Attitude controll interface](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/functional-areas/sensor-to-control/controllers/) of the drone, as specified in the dynamics model above. This interface takes **four control inputs**: `[collective thrust, roll, pitch, yaw]`. Thus, the action space is a [continuous box](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Box) represented as <b>float32</b>. The range for the thrust is <b>0.11264675 to 0.5933658</b>, while the orientations range from <b>-π/2 to π/2</b>. Actions needs to be a 2D array of shape `(n_envs, n_actions)`. For all exercises except the last one, we always have `n_envs=1` and `n_actions=4`.


> **Note**: There are different ways to represent orientations in 3D space. The observations of the environment returns [quaternions](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation), while the symbolic model works with Euler Angles. You do not have to worry too much about that, as you will use [this scipy package](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) to easily convert orientation representations.

> **Note**: The actions need to be a 2D array of shape `(n_envs, n_actions)`. Except for parallel simulations on GPU for learning based methods, we set `n_envs=1`.

Create an environment of type `DroneReachPos-v0`.

In [ ]:
# init env
import crazyflow  # register gymnasium envs
import gymnasium as gym
from gymnasium.wrappers.vector import JaxToNumpy

SEED = 42  # Seeding is very important for reproducibility; https://x.com/jakevdp/status/1247742792861757441

# Initialize the environment with various parameters.
env = gym.make_vec(
    "DroneReachPos-v0",
    num_envs=1,  # we will always use num_envs=1
    freq=500,
    device="cpu",  # running on CPU is sufficient
)

# Use a wrapper (https://gymnasium.farama.org/api/wrappers/) to convert the JAX array from the simulation to numpy arrays.
env = JaxToNumpy(env)

In [ ]:
# Get obs and act spaces
print(f"Observation space for a drone: {env.observation_space}")
# NOTE Note the limits for the action space, and recall their meanings! You will need it in further exercises
print(f"Action space for a drone: {env.action_space}")

Run the simulation using random control inputs.

Hint: You can use the `Tab`-Key to switch between cameras.

In [ ]:
# A simple simulation loop

# An examplary action for going up in attitude control
# We can use numpy arrays as we applied the JaxToNumpy wrapper.
# Recall that actions are of shape (1, 4)
action = np.zeros((1, 4), dtype=np.float32)
action[..., 0] = 0.4

obs, info = env.reset(seed=SEED)

# Step through the environment
for _ in range(1000):
    observation, reward, terminated, truncated, info = env.step(action)
    env.render()  # comment this out for much faster simulation
    env.unwrapped.sim.viewer.viewer.cam.lookat = env.unwrapped._goal[0]
    if terminated or truncated:
        env.reset()
env.close()
env.unwrapped.sim.close()

<div class="alert alert-success">
    <h3>Task 1: Exam Preparation</h3>
    <p>
    Why is the drone not only moving up in simulation but also in other directions, although the provided action vector points upwards?
    </p>
</div>

#### Symbolic model

As explained above, we provide the drone dynamics also as a [CasADi](https://web.casadi.org/) symbolic model. CasADi is a general-purpose tool for gradient-based numerical optimization – with a strong focus on optimal control. It implement algorithmic differentiation (AD), but also tools for numerical integration. Internally, of course, CasADi uses also an integrator to discretize the model. You will need the symbolic model in the following exercises to implement model-based control algorithms, such as MPC.

We implemented the convinience function `symbolic_from_sim` in [`crazyflow/sim/symbolic.py`](https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/sim/symbolic.py) to get a symbolic CasADi model with very similar behavior to the simulation. The continuous time dynamics model is stored in the [`fc_func`](https://github.com/utiasDSL/crazyflow/blob/bb429846791ee400dc18bddc3bed904ae4e6cb04/crazyflow/sim/symbolic.py#L72) property, and the discrete time dynamics in [`fd_func`](https://github.com/utiasDSL/crazyflow/blob/bb429846791ee400dc18bddc3bed904ae4e6cb04/crazyflow/sim/symbolic.py#L74). In the following, we demonstrate that the simulation and symbolic model indeed behave very similar. Slight differences arise due to numerical effects and different integrators used by our simulation and CasADi.

<div class="alert alert-success">
    <h3>Task 2: Check code.</h3>
    <p>
      To prepare for the next exercise, you can read the <code>__init__</code>, <code>setup_model</code> and <code>setup_linearization</code> methodes of the <a href="https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/sim/symbolic.py#L27"><code>SymbolicModel</code></a> class in <a href="https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/sim/symbolic.py"><code>crazyflow/sim/symbolic.py</code></a>. Understand how the symbolic representations of the drone’s dynamics, state, and cost functions are created. To understand this, you need some familarity with <a href="https://web.casadi.org/">CasADi</a> - now would be a good time to read up on this.
    </p>
</div>

<div class="alert alert-success">
    <h3>Task 3: Check code.</h3>
    <p>
      Check the implementation in the next code cell. You can play around with the identified parameters of the simulation model, and observe how the simulation and symbolic model diverge. You can do so by changing the parameters in the codeline after `TODO`.
    </p>
</div>

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from crazyflow.control.control import MAX_THRUST, MIN_THRUST
from crazyflow.sim.symbolic import symbolic_from_sim

# create env
envs = gym.make_vec("DroneReachPos-v0", num_envs=1, freq=500, device="cpu")
# obtain symbolic model
symbolic_model = symbolic_from_sim(envs.sim)

# TODO: change the parameters to see how the simulation and symbolic model diverge
crazyflow.sim.physics.SYS_ID_PARAMS["acc"] = np.array([20.907574256269616, 3.653687545690674])

# Prepare variables for logging
state_names = ["x", "y", "z", "x_dot", "y_dot", "z_dot"]
sim_log = {name: [] for name in state_names}
sym_log = {name: [] for name in state_names}

obs, info = envs.reset(seed=42)

pos = obs["pos"].squeeze()  # shape: (3,)
vel = obs["vel"].squeeze()  # shape: (3,)


init_state = np.array([*pos, 0, 0, 0, *vel, 0, 0, 0]) # only randomize pos and vel

sim_state = np.concatenate([pos, vel])

for i, name in enumerate(state_names):
    sim_log[name].append(sim_state[i])
    sym_log[name].append(sim_state[i])

xf = init_state
num_steps = 200
for i in range(num_steps):
    # Generate random input
    np.random.seed(seed=i)
    random_input = np.array(
        [
            np.random.uniform(4 * MIN_THRUST, 4 * MAX_THRUST),  # thrust
            np.random.uniform(-np.pi / 2, np.pi / 2),  # roll
            np.random.uniform(-np.pi / 2, np.pi / 2),  # pitch
            np.random.uniform(-np.pi / 2, np.pi / 2),  # yaw
        ],
        dtype=np.float32,
    ).reshape((1, 4))

    # Step (discretized) symbolic model
    res = symbolic_model.fd_func(x0=xf, p=random_input)
    xf = res["xf"].full().flatten()

    sym_state = np.concatenate([xf[:3], xf[6:9]])
    for j, name in enumerate(state_names):
        sym_log[name].append(sym_state[j])

    # Step simulation
    obs, reward, terminated, truncated, info = envs.step(random_input)
    pos = obs["pos"].squeeze()  # shape: (3,)
    vel = obs["vel"].squeeze()  # shape: (3,)
    sim_state = np.concatenate([pos, vel])

    for j, name in enumerate(state_names):
        sim_log[name].append(sim_state[j])

envs.close()
envs.unwrapped.sim.close()

# Plot results
time = np.arange(len(sym_log["x"]))
fig, axes = plt.subplots(2, 3, figsize=(16, 12))
fig.suptitle("Symbolic vs Simulation Model Comparison")
variables = [("x", "y", "z"), ("x_dot", "y_dot", "z_dot")]
colors = ["blue", "orange", "green"]


def plot_variable(ax, var_name, color="blue"):
    ax.plot(time, sym_log[var_name], label=f"{var_name}_symbolic", color=color)
    ax.plot(time, sim_log[var_name], label=f"{var_name}_sim", color=color, linestyle="--")
    ax.set_title(f"{var_name} trajectory")
    ax.set_xlabel("Time Steps")
    ax.set_ylabel(var_name)
    ax.legend()
    ax.grid(True)


for row, var_row in enumerate(variables):
    for col, var in enumerate(var_row):
        plot_variable(axes[row, col], var, colors[col])

plt.tight_layout()
plt.show()

## 4 PD Controller

Next, we implement a simple PD controller that enables the drone to fly to a target location and hover there. The PD controller takes as input a desired position (and velocity and yaw), as well as the controller gains `kp, kd`, and the current state of the drone. Note that the gymnasium environment returns the angular observations as [quaternions](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation). The controller returns the attitude command consisting of collective thrust, roll, pitch, and yaw, as explained above.

Hint: you can use the `Tab` key to switch between cameras.

<div class="alert alert-success">
    <h3>Task 4: Check code.</h3>
    <p>
      Go to the implementation of the PDDrone class in <code>exercise01/PD.py</code>. It inherits from a <code>BaseController</code> class that we will use throughout the following exercises.
    </p>
</div>

<div class="alert alert-success">
    <h3>Task 5: Exam preparation.</h3>
    <p>
    Play around with the <code>kp</code> and <code>kd</code> gains in the below code cell. In particular, observe what happens for high control gains? Do you think the action space limitations are chosen appropriately? You can also adjust the desired position, or the initial state (using the random seed or by providing custom initial conditions). Use the below code cells to generate plots.
    </p>
</div>

<div class="alert alert-success">
    <h3>Task 6: Exam preparation.</h3>
    <p>
    Change the drone mass used in the controller in the below code cell and observe the effect. How can the controller be improved to compensate for errors in the drone mass estimation?
    </p>
</div>

In [ ]:
import numpy as np
from crazyflow.constants import GRAVITY, MASS
from PD import PDDrone

# Logging
action_log = []

# TODO play around with different desired positions
desired_position = np.array([0.0, 0.0, 1.5])

# TODO play around with different controller gains. Specifically, try high gains and observe the effects of running into actuation limits.
# TODO change the drone mass used in the controller and observe the effect. How can the controller be improved to compensate for errors in the drone mass estimation?
controller = PDDrone(
    des_pos=desired_position,
    kp=np.array([0.4, 0.4, 1.25]),  # kp gain
    kd=np.array([0.2, 0.2, 0.5]),  # kd gain
    drone_mass=MASS,
)
# TODO play around with different initial conditions. You can also set specific initial conditions by passing the "options" dict to the env.reset() method: https://github.com/utiasDSL/crazyflow/blob/d87bc1eaf100e7d8927731c630e52a7163108ecf/crazyflow/gymnasium_envs/crazyflow.py#L162
observation, _ = env.reset(seed=SEED)


env.unwrapped._goal = env.unwrapped._goal.at[...].set(
    desired_position
)  # required for visualization of the goal marker

for i in range(1000):
    pos, vel, quat = observation["pos"].squeeze(), observation["vel"].squeeze(), observation["quat"].squeeze()
    action = controller.step_control(pos, vel, quat)
    action = np.clip(
        action, env.unwrapped.action_space.low, env.unwrapped.action_space.high
    )  # bound actions otherwise they are rejected by the step function
    observation, _, terminated, truncated, _ = env.step(action)

    # Logging
    action_log.append(action)

    # skip frames for faster rendering
    if i % 5 == 0:
        env.render()
        # focus camera on target
        env.unwrapped.sim.viewer.viewer.cam.lookat = env.unwrapped._goal[0]
    if terminated or truncated:
        env.reset()
env.close()
env.unwrapped.sim.close()

action_log = np.array(action_log)

print(f"Final position of the drone: {observation['pos']}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot results
steps = np.arange(len(action_log))
fig, axes = plt.subplots(1, 4, figsize=(20, 4))  # 1 row, 4 columns
fig.suptitle("Action logs")
variables = ["thrust", "roll", "pitch", "yaw"]


def plot_variable(ax, i):
    ax.plot(steps, action_log[:, 0, i], label=variables[i])
    ax.axhline(env.unwrapped.action_space.low[0][i], color="red", linestyle="--", label="min")
    ax.axhline(env.unwrapped.action_space.high[0][i], color="green", linestyle="--", label="max")
    if i == 0:
        ax.axhline(
            1.8 * controller.drone_mass * GRAVITY,
            color="black",
            linestyle="--",
            label="max (controller)",
        )
        ax.axhline(
            0.3 * controller.drone_mass * GRAVITY,
            color="grey",
            linestyle="--",
            label="min (controller)",
        )
    ax.set_xlabel("Time Steps")
    ax.set_ylabel(variables[i])
    ax.legend()
    ax.grid(True)


for i in range(4):
    plot_variable(axes[i], i)

plt.tight_layout()
plt.show()


>**Note**: for the plots you will observe maximal thrusts that get clipped before reaching the maximum thrust of 0.59 defined by the drone environment. The reason for that is that the thrust value gets clipped to even lower values in the PD controller implementation.

Great! Now you understand how to use the drone environment 🚀! In the coming exercises you will develop some cool controllers for it.

## 5 Example submission

Lastly for this introduction, we want you to get familiar with the python test cases and the submission system. **If you have not done so already, please carefully read the information in the `README.md` file regarding test cases and submissions**.

<div class="alert alert-info">
    <h3>Task 7: Implement return_true()</h3>
    <p>
      Please implement the function <code>return_true()</code> in <code>exercise01/return_true.py</code>. Run the corresponding tests for <img src="../resources/test_icon.png" alt="test_icon" width="20"/> <code>test/behavior/test_exercise01</code> locally on your computer to see if you implemented the function correctly. Try some faulty implementations as well, and observe the feedback you get. 
    </p>
    <p>
     When you pass all tests locally on your computer go ahead and submit to ARTEMIS. The tests should complete successfully on ARTEMIS as well.
    </p>
    <p>
    This task does not count towards your final grade (and thus doesn't include any hidden test cases).
    </p>
</div>

Execute the following cell as well to see the output of your implementation.

In [ ]:
%load_ext autoreload
%autoreload 2
from return_true import return_true

return_value = return_true()

print(f"return_value is{'' if isinstance(return_value, bool) else ' not'} a boolean.")
print(f"return_value is: {return_value}")

##### That's it!
You are done for this exercise. See you in the next one!